In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [ ]:
df_happy = pd.read_csv('hotel_happy_reviews.csv')
df_not_happy = pd.read_csv('hotel_not_happy_reviews.csv')

In [4]:
df_happy.head()

,User_ID,Description,Is_Response,hotel_ID
0,id10329,Stayed here with husband and sons on the way t...,happy,8
1,id10331,We had - rooms. One was very nice and clearly ...,happy,3
2,id10333,My wife & I stayed in this glorious city a whi...,happy,3
3,id10334,My boyfriend and I stayed at the Fairmont on a...,happy,7
4,id10336,"Steps off Times Square, nice rooms, stayed - n...",happy,3


In [5]:
df_not_happy.head()

,User_ID,Description,Is_Response,hotel_ID
0,id10326,The room was kind of clean but had a VERY stro...,not happy,3
1,id10327,I stayed at the Crown Plaza April -- - April -...,not happy,9
2,id10328,I booked this hotel through Hotwire at the low...,not happy,3
3,id10330,My girlfriends and I stayed here to celebrate ...,not happy,3
4,id10332,My husband and I have stayed in this hotel a f...,not happy,7


In [6]:
df_happy.shape, df_not_happy.shape

((26521, 4), (12411, 4))

In [25]:
#
# Source: https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21
#

import spacy
#
# !python -m spacy download en
#
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for t in tokens:
        if t.orth_.isspace():
            continue
        elif t.like_url:
            continue
        elif t.orth_.startswith('@'):
            continue
        else:
            lda_tokens.append(t.lower_)
    return lda_tokens


In [8]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vmullachery/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vmullachery/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    try:
        lemma = wn.morphy(word)
    except:
        return word
    if lemma:
        return lemma
    return word

from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)
en_stop = set(nltk.corpus.stopwords.words('english'))

In [27]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = list(filter(lambda x: True if len(x) > 4 else False, tokens))
    tokens = list(filter(lambda x: True if x not in en_stop else False, tokens))
    return [get_lemma(token) for token in tokens]

In [11]:
#all_text = np.append(df_happy.loc[:, 'Description'].values, df_not_happy.loc[:, 'Description'].values)
all_text = df_not_happy.loc[:, 'Description'].values
def get_text_data_as_tokens(text_array):
    tokens = []
    for text in text_array:
        c_tokens = prepare_text_for_lda(text)
        tokens += c_tokens,
    return tokens
text_data = get_text_data_as_tokens(all_text)

In [28]:
len(text_data), type(text_data)

(26521, list)

In [29]:
text_data[0]

[stayed,
 husband,
 alaska,
 cruise,
 loved,
 hotel,
 great,
 experience,
 north,
 tower,
 facing,
 north,
 views,
 floor,
 stunning,
 needle,
 cruise,
 ships,
 ordered,
 service,
 dinner,
 could,
 enjoy,
 perfect,
 views,
 service,
 dinners,
 delicious,
 perfect,
 everywhere,
 enjoy,
 almost,
 forgot-,
 heavenly,
 heavenly]

In [41]:
type(text_data[0][0])
sample_data='this is interesting and then some more fascinating'
sample_for_lda = prepare_text_for_lda(sample_data) #['interest', 'fascinate']
sample_for_lda, type(sample_for_lda)

(['interest', 'fascinate'], list)

In [53]:
from gensim import corpora
#dictionary = corpora.Dictionary([sample_for_lda])
dictionary = corpora.Dictionary(text_data)

#[type(t) for t in text_data[0]]

TypeError: decoding to str: need a bytes-like object, spacy.tokens.token.Token found

In [47]:
corpus = [dictionary.doc2bow(text) for text in text_data]

TypeError: decoding to str: need a bytes-like object, spacy.tokens.token.Token found

In [ ]:
import pickle
pickle.dump(corpus, )

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk

np.random.seed(7)

In [ ]:
from nltk.stem.snowball import EnglishStemmer
def lemmatize_and_stem(text):
    return EnglishStemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess_text(text):
    result = []
    for token in simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 3:
            result.append(lemmatize_and_stem(token))
    return result


In [ ]:
doc_sample = df_happy.loc[0, 'Description']
doc_sample
words = []
for word in doc_sample.split():
    words.append(word)
#print(words)
print(preprocess_text(doc_sample))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences